In [12]:
import cv2 as cv

import youtube_dl


In [13]:
import src.network as network
import src.file_helper as fh



weights = "yolov3.weights"
cfg = "yolov3.cfg"
classes = "coco.names"
net = network.load(weights, cfg)
classes = network.prepare_classes(classes)

output_layers = network.extract_output_layers(net)

In [14]:
url = "https://www.youtube.com/watch?v=zObiglKZKZc"

ydl_opts = {}

ydl = youtube_dl.YoutubeDL(ydl_opts)

info_dict = ydl.extract_info(url, download=False)

formats = info_dict.get('formats',None)


for f in formats:

    # I want the lowest resolution, so I set resolution as 144p
    if f.get('format_note',None) == '480p':

        #get the video url
        url = f.get('url',None)

        # open url with opencv
        cap = cv.VideoCapture(url)

        # check if url was opened
        if not cap.isOpened():
            print('video not opened')
            exit(-1)

        while True:
            # read frame
            ret, frame = cap.read()

            # check if frame is empty
            if not ret:
                break

            # display frame
            blob = network.image_to_blob(frame, 1/255)

            outputs = network.forward(net, blob, output_layers) 

            frame = network.postprocess(frame, outputs, classes, 0.8)

            cv.imshow('frame', frame)

            if cv.waitKey(30)&0xFF == ord('q'):
                break

        # release VideoCapture
        cap.release()

cv.destroyAllWindows()

[youtube] zObiglKZKZc: Downloading webpage
[youtube] zObiglKZKZc: Downloading video info webpage
0.4963203966617584
0.4830215573310852
0.37291258573532104
0.9975469708442688
0.9989938139915466
0.9545353651046753
0.4214111864566803
0.9594631791114807
0.5139687657356262
0.8961127996444702
0.8920008540153503
0.4518120288848877
0.9132021069526672
0.5638695955276489
0.4209116995334625
0.3203410506248474
0.2439405769109726
0.9267727136611938
0.42430466413497925
0.2765606641769409
0.2409934252500534
0.9159706830978394
0.6430476903915405
0.5113492012023926
0.4494363069534302
0.38863155245780945
0.26963937282562256
0.7849137783050537
0.7068344950675964
0.5837501883506775
0.9259470105171204
0.7767024040222168
0.769127368927002
0.6298366189002991
0.8928777575492859
0.7178096175193787
0.9460083246231079
0.7660276293754578
0.3102426826953888
0.23940284550189972
0.7328498959541321
0.6644613742828369
0.34270039200782776
0.7110223770141602
0.35224536061286926
0.2217305302619934
0.9739383459091187
0.88